In [4]:
import random
import time
import threading

# Predefined word list with meanings for hints
words = {
    "cat": "A small domesticated carnivorous mammal.",
    "tea": "A popular hot beverage.",
    "book": "A set of written or printed pages bound together.",
    "chair": "A piece of furniture for sitting.",
    "tree": "A woody perennial plant, typically having a single stem.",
    "umbrella": "A device for protection against rain or sun.",
    "python": "A high-level programming language.",
    "scramble": "To mix up or jumble.",
    "keyboard": "An input device for typing on a computer.",
    "butter": "A dairy product made from cream."
}

# Function to save to a file
PLAYER_FILE = "player_data.txt"

def load_player_data(player_name):
    try:
        with open(PLAYER_FILE, "r") as file:
            data = file.readlines()
        for line in data:
            name, games_played, best_time, words_guessed = line.strip().split('|')
            if name == player_name:
                return int(games_played), float(best_time), eval(words_guessed)
    except (FileNotFoundError, ValueError):
        pass
    return 0, float("inf"), []

def save_player_data(player_name, games_played, best_time, words_guessed):
    all_data = {}
    try:
        with open(PLAYER_FILE, "r") as file:
            data = file.readlines()
        for line in data:
            name, games_played_existing, best_time_existing, words_guessed_existing = line.strip().split('|')
            all_data[name] = (int(games_played_existing), float(best_time_existing), eval(words_guessed_existing))
    except FileNotFoundError:
        pass

    all_data[player_name] = (games_played, best_time, words_guessed)

    with open(PLAYER_FILE, "w") as file:
        for name, (games_played_entry, best_time_entry, words_guessed_entry) in all_data.items():
            file.write(f"{name}|{games_played_entry}|{best_time_entry}|{words_guessed_entry}\n")


# Function to get input with timeout
def input_with_timeout(prompt, timeout):
    result = [None]  # Use a mutable object to capture input

    def read_input():
        result[0] = input(prompt)

    input_thread = threading.Thread(target=read_input)
    input_thread.daemon = True
    input_thread.start()
    input_thread.join(timeout)

    if input_thread.is_alive():
        print("\n⏳ Time's up!")
        return None
    return result[0]

# Main game logic
def play_game(player_name):
    games_played, best_time, words_guessed = load_player_data(player_name)
    games_played += 1  # Increment games played
    
    print("\nSelect difficulty level:")
    print("1. Easy (Common short words)")
    print("2. Hard (Longer or complex words)")

    while True:
        difficulty = input("Enter your choice (1 for Easy, 2 for Hard): ")
        if difficulty in ["1", "2"]:
            break
        print("Invalid choice! Please select 1 (Easy) or 2 (Hard).")

    easy_words = [word for word in words.keys() if len(word) <= 4]
    hard_words = [word for word in words.keys() if len(word) > 4]
    word_list = easy_words if difficulty == "1" else hard_words

    original_word = random.choice(word_list)
    scrambled_word = "".join(random.sample(original_word, len(original_word)))

    print(f"\nScrambled word: {scrambled_word}")
    print("You have 30 seconds to guess the word!")

    attempts = 0
    time_limit = 30
    start_time = time.time()
    score = 0

    while True:
        remaining_time = time_limit - (time.time() - start_time)
        if remaining_time <= 0:
            print(f"\n⏳ Time's up! The correct word was '{original_word}'.")
            score = 0
            break

        guess = input_with_timeout(f"\n⏰ Time remaining: {int(remaining_time)} seconds. Your guess: ", remaining_time)
        if guess is None:
            print(f"\n⏳ Time's up! The correct word was '{original_word}'.")
            score = 0
            break

        if guess == original_word:
            elapsed_time = round(time.time() - start_time, 2)
            score = int(max(0, 100 - (elapsed_time * (100/30))))  # Score out of 100 based on time taken
    
            best_time = min(best_time, elapsed_time)
            words_guessed.append(original_word)
            save_player_data(player_name, games_played, best_time, words_guessed)

            print(f"\n🎉 Correct! You guessed the word in {elapsed_time} seconds.")
            break
        else:
            attempts += 1
            print("❌ Incorrect! Try again.")

            if attempts == 2:
                print(f"Hint: The first letter of the word is '{original_word[0]}'.")
            elif attempts == 3:
                print(f"Hint: The word means '{words[original_word]}'.")

    print(f"\nYour score: {score}")
    #save_score(player_name, best_time, score)
    save_player_data(player_name, games_played, best_time, words_guessed)

def display_top_scores():
        try:
            with open("top_scores.txt", "r") as file:
                scores = file.readlines()
                if scores:
                    print("\n--- Top Scores ---")
                    for score in scores:
                        print(score.strip())
                else:
                    print("No scores available yet.")
        except FileNotFoundError:
            print("No scores found. Play the game to record your score.")
# Main program
def main():
    print("\n--- Word Scramble Game ---")
    
    player_name = input("Enter your name: ")  # Get the player's name here
    
    while True:
        print("\n1. Play Game")
        print("2. View Top Scores")
        print("3. Quit")
        choice = input("Enter your choice: ")
        if choice == "1":
            play_game(player_name)
        elif choice == "2":
            display_top_scores()
        elif choice == "3":
            print("Thank you for playing! Goodbye!")
            break
        else:
            print("Invalid choice! Please enter 1, 2, or 3.")
# Run the game
if __name__ == "__main__":
    main()
    


--- Word Scramble Game ---
Enter your name: jawad

1. Play Game
2. View Top Scores
3. Quit
Enter your choice: 1

Select difficulty level:
1. Easy (Common short words)
2. Hard (Longer or complex words)
Enter your choice (1 for Easy, 2 for Hard): 2

Scrambled word: ihacr
You have 30 seconds to guess the word!

⏰ Time remaining: 30 seconds. Your guess: chair

🎉 Correct! You guessed the word in 11.56 seconds.

Your score: 61

1. Play Game
2. View Top Scores
3. Quit
Enter your choice: 1

Select difficulty level:
1. Easy (Common short words)
2. Hard (Longer or complex words)
Enter your choice (1 for Easy, 2 for Hard): 1

Scrambled word: bkoo
You have 30 seconds to guess the word!

⏰ Time remaining: 30 seconds. Your guess: book

🎉 Correct! You guessed the word in 3.02 seconds.

Your score: 89

1. Play Game
2. View Top Scores
3. Quit


KeyboardInterrupt: Interrupted by user